# FD2425
Jorge Machado 21181


Imports

In [29]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import tkinter as tk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn import metrics
from sklearn.svm import LinearSVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, cross_val_score
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, median_absolute_error,r2_score
from sklearn.decomposition import PCA

Open dataset

Lazy Predictor Regressor

In [ ]:
# Load the dataset with the specified columns
df = pd.read_csv('./cars.csv')

# Remove rows with NaN in any column
df = df.dropna().copy()

# Convert categorical variables to dummy variables
df = pd.get_dummies(df, drop_first=True)

# Sample the data (e.g., 10% of the original data)
df_sample = df.sample(frac=0.5, random_state=50)

# Split the data into features and target variable
X = df_sample.drop(['price_usd'], axis=1)
y = np.log1p(df_sample['price_usd'])  # Use log-transformed target for training

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize LazyRegressor
reg = LazyRegressor()

# Fit and evaluate models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# Display all the evaluated models
print(models)

# Supervised Learning

Regression = learn f with continuous output values -> car value

In [30]:
# Load the dataset with the specified columns
df = pd.read_csv('./cars.csv')

""" df = pd.read_csv('./cars.csv', usecols=['manufacturer_name', 'model_name', 'transmission', 'color', 
                                        'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas', 
                                        'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 
                                        'state', 'drivetrain', 'price_usd', 'is_exchangeable']) """

# Remove rows with NaN in any column
df = df.dropna().copy()

df.head()

print("Min price:", df['price_usd'].min())
print("Max price:", df['price_usd'].max())
print("Mean price:", df['price_usd'].mean())
print("Standard deviation of price:", df['price_usd'].std())

Min price: 1.0
Max price: 50000.0
Mean price: 6637.160884193038
Standard deviation of price: 6425.19899630064


In [31]:
print(df.describe(), "\n")
print(df.shape, "\n\n")
print (df.columns, "\n\n")
print(df.isnull().sum())

       odometer_value  year_produced  engine_capacity  price_usd  \
count        38521.00       38521.00         38521.00   38521.00   
mean        248910.07        2002.94             2.06    6637.16   
std         136059.50           8.06             0.67    6425.20   
min              0.00        1942.00             0.20       1.00   
25%         158000.00        1998.00             1.60    2100.00   
50%         250000.00        2003.00             2.00    4800.00   
75%         325000.00        2009.00             2.30    8950.00   
max        1000000.00        2019.00             8.00   50000.00   

       number_of_photos  up_counter  duration_listed  
count          38521.00    38521.00         38521.00  
mean               9.65       16.31            80.58  
std                6.09       43.29           112.84  
min                1.00        1.00             0.00  
25%                5.00        2.00            23.00  
50%                8.00        5.00            59.00  
75

**First Tests**

In [32]:
# Define numerical and categorical columns
numerical_cols = ['price_usd', 'year_produced', 'odometer_value', 'engine_capacity', 'duration_listed', 'number_of_photos']
categorical_cols = ['model_name', 'has_warranty', 'is_exchangeable', 'transmission', 'color', 'engine_fuel', 'state', 'drivetrain', 'engine_type', 'body_type', 'engine_has_gas', 'manufacturer_name']

# Apply StandardScaler to numerical columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [33]:
# Feature selection for modeling
X = df[['manufacturer_name', 'model_name', 'year_produced', 'odometer_value', 'color', 'transmission', 'engine_fuel', 'engine_type',
        'engine_capacity', 'body_type', 'has_warranty', 'state', 'drivetrain', 'is_exchangeable']]

df['log_price_usd'] = np.log1p(df['price_usd'])
y = df['log_price_usd']

# Encode categorical features
X = pd.get_dummies(X, columns=['manufacturer_name', 'model_name', 'year_produced', 'odometer_value', 'color', 'transmission', 'engine_fuel', 'engine_type',
                               'engine_capacity', 'body_type', 'has_warranty', 'state', 'drivetrain', 'is_exchangeable'], drop_first=True)

In [ ]:
# Split the data into 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split the 80% training data into 75% training and 25% validation (which is 60% training and 20% validation of the original data)
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Replace spaces in feature names with underscores
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')
x_train.columns = x_train.columns.str.replace(' ', '_')
x_val.columns = x_val.columns.str.replace(' ', '_')



def calculate_metrics(y_test, y_pred):
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE in percentage
    return r2, mae, mse, rmse, mape




for i in range(1, 4):
    if i == 1:
        model = LGBMRegressor()
        model_name = "LGBMRegressor"
    elif i == 2:
        model = HistGradientBoostingRegressor()
        model_name = "HistGradientBoostingRegressor"
    elif i == 3:
        model = GradientBoostingRegressor()
        model_name = "GradientBoostingRegressor"
        
    mask = ~np.isnan(y_train)
    x_train = x_train[mask]
    y_train = y_train[mask]

    # Check for NaN values in y_train before fitting
    if np.isnan(y_train).any():
        print(f"NaN values found in y_train for {model_name}. Skipping this model.")
        continue

    # Fit the model
    model.fit(x_train, y_train)

    # Predict on the test set
    y_pred = np.expm1(model.predict(X_test))

    # Clean NaN values from y_test and y_pred
    nan_indices = np.isnan(y_test) | np.isnan(y_pred)
    y_test_clean = y_test[~nan_indices]
    y_pred_clean = y_pred[~nan_indices]

    # Calculate metrics
    r2, mae, mse, rmse, mape = calculate_metrics(y_test_clean, y_pred_clean)


    fig, ax = plt.subplots()
    ax.scatter(X_test.iloc[:, 0], y_pred, c='r', marker='*')  # Use .iloc for DataFrames
    ax.set_xlabel("idk")
    ax.set_ylabel("idk")
    ax.set_title("Predicted Values")

    plt.show()


    # Print results
    print(f"Resultado do {model_name}\n")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.4f}%")

NaN values found in y_train for LGBMRegressor. Skipping this model.
NaN values found in y_train for HistGradientBoostingRegressor. Skipping this model.
NaN values found in y_train for GradientBoostingRegressor. Skipping this model.


**Second Tests**

In [ ]:
# Feature selection for modeling
X = df[['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color', 'transmission', 'engine_fuel',
       'engine_capacity', 'state', 'drivetrain', 'is_exchangeable' ]]

y = df['price_usd']  # Target variable

# Apply log transformation to the target variable
df['log_price_usd'] = np.log1p(df['price_usd'])



# Encode categorical features
X = pd.get_dummies(X, columns=['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color', 'transmission', 'engine_fuel',
       'engine_capacity', 'state', 'drivetrain', 'is_exchangeable' ], drop_first=True)



# Split the data into training and testing sets without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.60, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Replace spaces in feature names with underscores
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')
x_train.columns = x_train.columns.str.replace(' ', '_')
x_val.columns = x_val.columns.str.replace(' ', '_')


model = LGBMRegressor()

model.fit(x_train, y_train)


y_pred = np.expm1(model.predict(X_test))  # Convert predictions back to USD


# Check for NaN values in y_test and y_pred
nan_indices = np.isnan(y_test) | np.isnan(y_pred)

# Remove NaN values
y_test_clean = y_test[~nan_indices]
y_pred_clean = y_pred[~nan_indices]

# Calculate r2_score
r2 = r2_score(y_test_clean, y_pred_clean)
mae = mean_absolute_error(y_test_clean, y_pred_clean)
mse = mean_squared_error(y_test_clean, y_pred_clean)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_clean - y_pred_clean) / y_test_clean)) * 100  # MAPE in percentage

print ("Resultado do LGBMRegressor\n")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

**Thrid Tests** 

In [ ]:
df = df.dropna().copy()

# Feature selection
X = df[['manufacturer_name', 'model_name', 'year_produced', 'odometer_value', 'color', 'transmission', 'engine_fuel', 
        'engine_capacity', 'drivetrain']]

# Target variable with log transformation
y = np.log1p(df['price_usd'])

# Encode categorical features correctly (avoid encoding numeric features)
categorical_features = ['manufacturer_name', 'model_name', 'color', 'transmission', 'engine_fuel', 'drivetrain']
X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Train-test split (correct order)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.6, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Replace spaces in feature names with underscores
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')
x_train.columns = x_train.columns.str.replace(' ', '_')
x_val.columns = x_val.columns.str.replace(' ', '_')


# Initialize model
model = LGBMRegressor()

# Fit model on training set
model.fit(X_train, y_train)

# Predict on test set
y_pred = np.expm1(model.predict(X_test))  # Convert predictions back to USD

# Check for NaN values in y_test and y_pred
nan_indices = np.isnan(y_test) | np.isnan(y_pred)

# Remove NaN values
y_test_clean = y_test[~nan_indices]
y_pred_clean = y_pred[~nan_indices]

# Calculate r2_score
r2 = r2_score(y_test_clean, y_pred_clean)
mae = mean_absolute_error(y_test_clean, y_pred_clean)
mse = mean_squared_error(y_test_clean, y_pred_clean)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test_clean - y_pred_clean) / y_test_clean)) * 100  # MAPE in percentage

# Print results
print("\n\nResultado do LGBMRegressor\n")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

**Fourth Tests**

In [ ]:
# Feature selection for modeling
X = df[['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color' ]]

#y = df['price_usd']  # Target variable

# Apply log transformation to the target variable
y =df['log_price_usd'] = np.log1p(df['price_usd'])

# Encode categorical features
X = pd.get_dummies(X, columns=['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color'  ], drop_first=True)





# Split the data into training and testing sets without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.60, random_state=42)


x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# Replace spaces in feature names with underscores
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')
x_train.columns = x_train.columns.str.replace(' ', '_')
x_val.columns = x_val.columns.str.replace(' ', '_')


model = LGBMRegressor()


# Fit the model using the correct training split
model.fit(x_train, y_train)

# Evaluate using X_test (testing set)
y_pred = np.expm1(model.predict(X_test))  # Convert predictions back to USD


r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE in percentage

print ("\n\nResultado do LGBMRegressor\n")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

**Fith Tests**

In [ ]:
# Feature selection for modeling
X = df[['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color', 'transmission' ]]

y = df['price_usd']  # Target variable

# Apply log transformation to the target variable
y= df['log_price_usd'] = np.log1p(df['price_usd'])

# Encode categorical features
X = pd.get_dummies(X, columns=['manufacturer_name','model_name', 'year_produced', 'odometer_value', 'color'  ], drop_first=True)

# Split the data into training and testing sets without stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.60, random_state=42)


x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



# Replace spaces in feature names with underscores
X_train.columns = X_train.columns.str.replace(' ', '_')
X_test.columns = X_test.columns.str.replace(' ', '_')
x_train.columns = x_train.columns.str.replace(' ', '_')
x_val.columns = x_val.columns.str.replace(' ', '_')


model = LGBMRegressor()


# Fit the model using the correct training split
model.fit(x_train, y_train)

# Evaluate using X_test (testing set)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE in percentage

print ("\n\nResultado do LGBMRegressor\n")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.4f}%")

Metricas utilizadas para a regressão

# MSE – Mean Squared error [1]
Mede a diferença média quadrática entre os valores previstos e os valores reais no conjunto de dados.

**Vantagens:**

- Fornece uma medida abrangente da precisão do modelo.
- Sensível tanto a grandes quanto a pequenos erros.
- Fácil de calcular e interpretar.

**Limitações:**

- Pode ser fortemente influenciado por outliers.
- Penaliza erros grandes de forma desproporcional, o que pode nem sempre ser desejável.

# MAE – Mean Absolute Error [2]
O Erro Médio Absoluto calcula a diferença média entre os valores calculados e os valores reais.

**Vantagens:**

- Interpretabilidade: MAE fornece uma medida direta do erro médio, calculando a diferença absoluta média entre os valores previstos e os valores reais. Por exemplo, um MAE de 5 indica que as previsões, em média, desviam-se em 5 unidades dos valores reais.
- Robustez a Outliers: MAE é resiliente a outliers, tratando todos os erros de forma igual. Ao contrário do Erro Quadrático Médio (MSE), que penaliza discrepâncias maiores de forma mais intensa, o MAE mantém estabilidade e confiabilidade quando os conjuntos de dados incluem outliers.
- Aplicação Prática: MAE é útil em cenários do mundo real onde os custos de erro são lineares, como previsão de demanda ou estimativa de preços de imóveis. Ele oferece insights sobre os tamanhos médios dos erros, tornando-se uma métrica preferida para avaliar a qualidade da previsão com base no erro absoluto em vez do erro relativo.
- Penalidade Linear: A natureza linear do MAE significa que o impacto de cada erro na aprendizagem do modelo é diretamente proporcional à sua magnitude. Isso permite que o modelo minimize os erros médios sem focar desproporcionalmente em discrepâncias maiores, garantindo um desempenho equilibrado em todos os pontos de dados.

# MAPE – Mean Absolute Percentage Error [3]
- O termo MAPE determina quão melhor é a precisão da nossa previsão.
- O melhor valor para MAPE é 0, enquanto um valor mais alto indica que as previsões não são precisas o suficiente. No entanto, o quão grande deve ser um valor MAPE para ser considerado uma previsão ineficiente depende do caso de uso.

# MDAE – Median Absolute Error [4]
A principal vantagem de usar essa métrica é sua forte resiliência a outliers.

[1] `https://www.geeksforgeeks.org/mean-squared-error/`  
[2] `https://www.geeksforgeeks.org/how-to-calculate-mean-absolute-error-in-python/`  
[3] `https://www.geeksforgeeks.org/how-to-calculate-mape-in-python/`  
[4] `https://insidelearningmachines.com/median_absolute_error/` 

In [ ]:
# Ensure the same columns are present in the training and testing sets
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error

X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Function to apply PCA and train the regressor model
def apply_pca_and_train_regressor(X_train, X_test, y_train, y_test, n_components):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Train the regressor model (example: LGBMRegressor)
    model = LGBMRegressor(random_state=42)
    model.fit(X_train_pca, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test_pca)
    print(f"Results for PCA with {n_components} components:")
    print(f"R-Squared: {r2_score(y_test, y_pred):.2f}")
    print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")
    print("\n")

# Apply PCA with different components and train the model
apply_pca_and_train_regressor(X_train, X_test, y_train, y_test, n_components=0.95)
apply_pca_and_train_regressor(X_train, X_test, y_train, y_test, n_components=10)
apply_pca_and_train_regressor(X_train, X_test, y_train, y_test, n_components=20)

Lazy predictor for regression

Para esta quantidade de features usecols=['manufacturer_name', 'model_name', 'transmission', 'color', 
                                        'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas', 
                                        'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 
                                        'state', 'drivetrain', 'price_usd', 'is_exchangeable']


e utilizando o lazy regressor, o melhor modelo para este caso foi o LGBMRegressor, no entanto os valores relativos ao RMSE não foram muito positivos visto que 0.45 é um valor alto.

Então o passo seguinte foi pegar neste modelo e testar com diferentes features para averiguar qual é o melhor conjunto de features para este problema.

In regression models, good values for evaluation metrics indicate how well the model is performing. Here are some common metrics and their ideal values:

**R-squared (R²):**

Definition: Proportion of the variance in the dependent variable that is predictable from the independent variables.
Good Value: Closer to 1. An R² value of 1 indicates that the model explains all the variability of the response data around its mean.
Adjusted R-squared:

Definition: Adjusted version of R² that accounts for the number of predictors in the model.
Good Value: Closer to 1. It adjusts for the number of predictors, so it is more reliable for comparing models with different numbers of predictors.

**Mean Absolute Error (MAE):**

Definition: Average of the absolute errors between predicted and actual values.
Good Value: Closer to 0. Lower values indicate better model performance.
Mean Squared Error (MSE):

Definition: Average of the squared errors between predicted and actual values.
Good Value: Closer to 0. Lower values indicate better model performance.


**Root Mean Squared Error (RMSE):**

Definition: Square root of the average of the squared errors between predicted and actual values.
Good Value: Closer to 0. Lower values indicate better model performance.
Mean Absolute Percentage Error (MAPE):

Definition: Average of the absolute percentage errors between predicted and actual values.
Good Value: Closer to 0%. Lower values indicate better model performance.

**Akaike Information Criterion (AIC):**

Definition: Measure of the relative quality of a statistical model for a given set of data.
Good Value: Lower values are better. It balances model fit and complexity.

**Bayesian Information Criterion (BIC):**

Definition: Similar to AIC but with a higher penalty for models with more parameters.
Good Value: Lower values are better. It also balances model fit and complexity.
In general, the ideal values for these metrics depend on the specific context and the nature of the data. Comparing these metrics across different models can help you choose the best model for your regression task.

utilizar diferentes pca
para classificacao fazer histograma com os diferentes preços de carros e criar classes com base num range de preços por exemplo de 0 a 10000, 10001 a 20000, etc.